In [1]:
# %load imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions
from datetime import datetime
CHROME_DRIVER = './chromedriver'
FIREFOX_DRIVER = './geckodriver'
import time
import re
import requests
import json
import random
import copy
import pprint


def setup_driver(headless=True, driver_type=CHROME_DRIVER) -> webdriver:
    if 'chrome' in driver_type.lower():
        options = ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)
    else:
        options = FirefoxOptions()
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Firefox(executable_path=driver_type, firefox_options=options)
    return driver


In [2]:
def get_soup(link):
    driver = setup_driver()
    driver.get(link)
    time.sleep(8)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    return soup

In [3]:
page = get_soup('https://www.indeed.com/jobs?q=%22Software+Engineer%22&l=')

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


In [46]:
def link_to_other_pages(page):
    other_page_links = []

lins = page.find('ul',{'class':'pagination-list'}).find_all('li')
for lin in lins:
    if lin.find('a', href = True):
        print(lin.find('a')['href'])
    

/jobs?q=%22Software+Engineer%22&start=10
/jobs?q=%22Software+Engineer%22&start=20
/jobs?q=%22Software+Engineer%22&start=30
/jobs?q=%22Software+Engineer%22&start=40
/jobs?q=%22Software+Engineer%22&start=10


In [29]:
other_page_links = []

for link in

<html dir="ltr" lang="en"><head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script async="" src="https://www.google-analytics.com/plugins/ua/linkid.js" type="text/javascript"></script><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script async="" src="//www.google-analytics.com/analytics.js"></script><script id="polyfill-script-bundle">/* Disable minification (remove `.min` from URL path) for more info */

</script>
<script src="//d3fw5vlhllyvee.cloudfront.net/s/085ee87/en_US.js" type="text/javascript"></script>
<link href="//d3fw5vlhllyvee.cloudfront.net/s/c10a675/jobsearch_all.css" rel="stylesheet" type="text/css"/>
<link href="https://rss.indeed.com/rss?q=%22Software+Engineer%22" rel="alternate" title="Software Engineer Jobs, Employment" type="application/rss+xml"/>
<link href="/m/jobs?q=%22Software+Engineer%22" media="only screen and (max-width: 640px)" rel="alternate"/>
<script type="text/javascript">

if (typeof window['closu

In [5]:
container = page.find('div',{'id':'mosaic-provider-jobcards'})

In [9]:
boxes = container.find_all('a', href =True)

In [16]:
links_to_postings = []
for box in boxes:
    first = box['href']
    if '/rc/clk?jk' in first:
        link = first.replace('/rc/clk?','https://www.indeed.com/viewjob?')
        links_to_postings.append(link)

In [17]:
soups = []
for link in links_to_postings:
    soup = get_soup(link)
    soups.append(soup)

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


In [28]:
for soup in soups:
    title = soup.find('h1',{'class':'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).text
    
    original = soup.find('div',{'id':'originalJobLinkContainer'}).find('a')['href']
    print(original)

https://www.indeed.com/rc/clk?jk=835c0973db3a626b&from=vj&pos=top&mvj=0&sjdu=YmZE5d5THV8u75cuc0H6Y26AwfY51UOGmh3Z9h4OvXiOEkPw4KYhugLMUqLWndNyXAwdulcUk0atwlDdDDqlBQ&astse=f208cfae285d96f8&assa=8678
https://www.indeed.com/rc/clk?jk=a4a0e9ccf8a052a8&from=vj&pos=top&mvj=0&sjdu=YmZE5d5THV8u75cuc0H6Y26AwfY51UOGmh3Z9h4OvXhH58D9m5sL0BMHjUFCdf4ml8EZNoFLzyHkpvcd7-Y9bg&astse=029ef09574bc52c6&assa=3253
https://www.indeed.com/rc/clk?jk=35fada2cdf3bb457&from=vj&pos=top&mvj=0&sjdu=YmZE5d5THV8u75cuc0H6Y26AwfY51UOGmh3Z9h4OvXh6LZ8vjpXG-vi9tSaCeqXjXEqzEHpaMck04qufWYB5sg&astse=30a3af993b272ba8&assa=4847
https://www.indeed.com/rc/clk?jk=c18ba28320ca805c&from=vj&pos=top&mvj=0&sjdu=YmZE5d5THV8u75cuc0H6Y26AwfY51UOGmh3Z9h4OvXiybmhzur9Yc5NTE9JskM4PkiS-2XS_43VyhyRGICpJaw&astse=d6a071bb88ebdfad&assa=6437
https://www.indeed.com/rc/clk?jk=ae3139c2282ce2b2&from=vj&pos=top&mvj=0&sjdu=YmZE5d5THV8u75cuc0H6Y26AwfY51UOGmh3Z9h4OvXhRiHMJvVdF-Pav5iy2cXr3MIfcMu539kP3i1FMxIq2rA&astse=134d802cad2cc9fe&assa=8902
https://www.ind